# Asymmetric Grs-Pyr Binary Garnet soln (van-Laar)
- based on Holland2018

In [ ]:
import numpy as np
import scipy as sp
import pandas as pd
from scipy import spatial
from scipy import interpolate
import matplotlib.pyplot as plt

import vanlaar

## Create Pyrope-Grossular Binary Soln models
- uses parameter values from Holland2018

In [ ]:
R = 8.3145

# Pressure appears to be in kbars???

Wh = 45.4*1e3
Ws = .01*1e3
Wv = .04*1e3

alpha = 2.5
mult = 3


In [ ]:
regular = vanlaar.RegularSoln(Wh, Wv=Wv, Ws=Ws, mult=mult)
asymm = vanlaar.AsymmSoln(Wh, Wv=Wv, Ws=Ws, alpha=alpha, mult=mult)

In [ ]:
P=0
T0_crit = 750

T = T0_crit*.9

## Compare excess gibss energies

In [ ]:
X = np.linspace(0,1)

G_XS = regular.gibbs_excess(X, T=T, P=P)
G_asymm_XS = asymm.gibbs_excess(X, T=T, P=P)

plt.figure()
plt.plot(X, G_XS, 'k--')

plt.plot(X, G_asymm_XS, 'r-')

#G_XS

## Find solvus using common tangent

In [ ]:
regular.test_solvus_finder(T=T, P=P)

In [ ]:
asymm.test_solvus_finder(T=T, P=P)

# Calculate a-X relation for supercritical solution 
- calculate for temperature above miscibility gap closing

- NOTE: DEBUG FIX needed for multiplicity effect on activities

In [ ]:
def plot_aX(mod, label, fmt, T=300, P=1, figure=None, N=101):
    if figure is None:
        figure = plt.figure()
        
    X = np.linspace(0,1,N)

    a1 = mod.activity(X, T, P=P, endmem=1)
    a2 = mod.activity(X, T, P=P, endmem=2)
    
    plt.plot(X, a1, fmt,label=label)
    plt.plot(X, a2, fmt)

    plt.plot(X, X  , ':',color='gray')
    plt.plot(X, 1-X, ':',color='gray')

    plt.xlim(0,1)
    plt.xlabel('X')
    plt.ylabel('a')
    plt.legend()
    return figure

In [ ]:
T_supcrit = T0_crit*1.2

figure = plot_aX(regular, 'regular', 'k--', T=T_supcrit)
figure = plot_aX(asymm, 'van laar', 'r-', T=T_supcrit, figure=figure)

# Solvus Calculation

In [ ]:
def plot_solvus(T, X_solvus, label='', fmt='k--', figure=figure):

    if figure is None:
        figure = plt.figure()
        
    plt.plot(X_solvus[:,0], T, fmt, label=label)
    plt.plot(X_solvus[:,1], T, fmt)

    plt.xlim(-.01,1.01)
    plt.xlabel('X')
    plt.ylabel('T [K]')
    
    if label!='':
        plt.legend()
        
    return figure

In [ ]:
# T = np.linspace(.5*T0_crit,3*T0_crit,301)
T = np.linspace(400,870, 3601)
P0=1e-3

N=6001
upsamp_fac = 1
allow_interp = False
Phi=30

In [ ]:
# X_solvus = regular.solvus(T, P=P0)
X_solvus_asymm = asymm.solvus(T, P=P0, N=N, allow_interp=allow_interp)

# X_solvus_hi = regular.solvus(T, P=Phi)
X_solvus_asymm_hi = asymm.solvus(T, P=Phi, N=N, allow_interp=allow_interp)

In [ ]:
def reformat_solvus(X_solvus, T):
    Xsolv = np.hstack((X_solvus[:,0],X_solvus[::-1,1]))
    Tsolv = np.hstack((T, T[::-1]))
    # ind_solv = ~(np.isnan(Tsolv) or np.isnan(Xsolv))
    ind_solv = ~(np.isnan(Xsolv)|np.isnan(Tsolv))

    Xsolv = Xsolv[ind_solv]
    Tsolv = Tsolv[ind_solv]
    
    return Xsolv, Tsolv



In [ ]:
Xsolv, Tsolv = reformat_solvus(X_solvus_asymm,  T)
Xsolv_hi, Tsolv_hi = reformat_solvus(X_solvus_asymm_hi,  T)

In [ ]:
df_solvus = pd.DataFrame({'X':Xsolv, 'T':Tsolv})
df_solvus_3 = pd.DataFrame({'X':Xsolv_hi, 'T':Tsolv_hi})
df_solvus.to_csv('data/solvus_Holland2018.csv')
df_solvus_3.to_csv('data/solvus_3_Holland2018.csv')

In [ ]:
plt.figure()
plt.plot(Xsolv, Tsolv,  'k-')
plt.plot(Xsolv_hi, Tsolv_hi,  'r-')

In [ ]:
# figure = plot_solvus(T, X_solvus, label='regular', fmt='k--')
figure = plot_solvus(T, X_solvus_asymm, label='van laar', fmt='r-', figure=figure)

# figure = plot_solvus(T, X_solvus_hi, label='regular (8 GPa)', fmt='k-.')
figure = plot_solvus(T, X_solvus_asymm_hi, label='van laar (3 GPa)', fmt='r:', figure=figure)
plt.title('Grossular-Pyrope Binary')

# plt.ylim(780,800)
# plt.xlim(.65,.87)